# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa5ecca0f40>
├── 'a' --> tensor([[ 0.8140, -0.0181,  0.4719],
│                   [-0.6035,  1.3166,  0.2733]])
└── 'x' --> <FastTreeValue 0x7fa5ecca0f10>
    └── 'c' --> tensor([[ 0.6333, -1.7667, -0.8784,  0.2224],
                        [-1.5923,  0.0482, -1.1485,  1.2433],
                        [ 0.1746,  0.1556,  0.4432,  0.4610]])

In [4]:
t.a

tensor([[ 0.8140, -0.0181,  0.4719],
        [-0.6035,  1.3166,  0.2733]])

In [5]:
%timeit t.a

58.6 ns ± 0.0549 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa5ecca0f40>
├── 'a' --> tensor([[-0.1420,  0.7053,  0.8932],
│                   [ 0.5154, -0.5341, -0.2231]])
└── 'x' --> <FastTreeValue 0x7fa5ecca0f10>
    └── 'c' --> tensor([[ 0.6333, -1.7667, -0.8784,  0.2224],
                        [-1.5923,  0.0482, -1.1485,  1.2433],
                        [ 0.1746,  0.1556,  0.4432,  0.4610]])

In [7]:
%timeit t.a = new_value

60.7 ns ± 0.0154 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8140, -0.0181,  0.4719],
               [-0.6035,  1.3166,  0.2733]]),
    x: Batch(
           c: tensor([[ 0.6333, -1.7667, -0.8784,  0.2224],
                      [-1.5923,  0.0482, -1.1485,  1.2433],
                      [ 0.1746,  0.1556,  0.4432,  0.4610]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8140, -0.0181,  0.4719],
        [-0.6035,  1.3166,  0.2733]])

In [11]:
%timeit b.a

55.4 ns ± 0.0268 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8685, -0.0156,  0.0111],
               [-0.8088,  0.1826,  0.9408]]),
    x: Batch(
           c: tensor([[ 0.6333, -1.7667, -0.8784,  0.2224],
                      [-1.5923,  0.0482, -1.1485,  1.2433],
                      [ 0.1746,  0.1556,  0.4432,  0.4610]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.182 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

847 ns ± 18.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 35.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 536 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 489 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa536f6feb0>
├── 'a' --> tensor([[[ 0.8140, -0.0181,  0.4719],
│                    [-0.6035,  1.3166,  0.2733]],
│           
│                   [[ 0.8140, -0.0181,  0.4719],
│                    [-0.6035,  1.3166,  0.2733]],
│           
│                   [[ 0.8140, -0.0181,  0.4719],
│                    [-0.6035,  1.3166,  0.2733]],
│           
│                   [[ 0.8140, -0.0181,  0.4719],
│                    [-0.6035,  1.3166,  0.2733]],
│           
│                   [[ 0.8140, -0.0181,  0.4719],
│                    [-0.6035,  1.3166,  0.2733]],
│           
│                   [[ 0.8140, -0.0181,  0.4719],
│                    [-0.6035,  1.3166,  0.2733]],
│           
│                   [[ 0.8140, -0.0181,  0.4719],
│                    [-0.6035,  1.3166,  0.2733]],
│           
│                   [[ 0.8140, -0.0181,  0.4719],
│                    [-0.6035,  1.3166,  0.2733]]])
└── 'x' --> <FastTreeValue 0x7fa536f6fac0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 37.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa53dbdaf70>
├── 'a' --> tensor([[ 0.8140, -0.0181,  0.4719],
│                   [-0.6035,  1.3166,  0.2733],
│                   [ 0.8140, -0.0181,  0.4719],
│                   [-0.6035,  1.3166,  0.2733],
│                   [ 0.8140, -0.0181,  0.4719],
│                   [-0.6035,  1.3166,  0.2733],
│                   [ 0.8140, -0.0181,  0.4719],
│                   [-0.6035,  1.3166,  0.2733],
│                   [ 0.8140, -0.0181,  0.4719],
│                   [-0.6035,  1.3166,  0.2733],
│                   [ 0.8140, -0.0181,  0.4719],
│                   [-0.6035,  1.3166,  0.2733],
│                   [ 0.8140, -0.0181,  0.4719],
│                   [-0.6035,  1.3166,  0.2733],
│                   [ 0.8140, -0.0181,  0.4719],
│                   [-0.6035,  1.3166,  0.2733]])
└── 'x' --> <FastTreeValue 0x7fa5eccf8e50>
    └── 'c' --> tensor([[ 0.6333, -1.7667, -0.8784,  0.2224],
                        [-1.5923,  0.0482, -1.1485,  1.2433],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60 µs ± 28.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.8140, -0.0181,  0.4719],
                [-0.6035,  1.3166,  0.2733]],
       
               [[ 0.8140, -0.0181,  0.4719],
                [-0.6035,  1.3166,  0.2733]],
       
               [[ 0.8140, -0.0181,  0.4719],
                [-0.6035,  1.3166,  0.2733]],
       
               [[ 0.8140, -0.0181,  0.4719],
                [-0.6035,  1.3166,  0.2733]],
       
               [[ 0.8140, -0.0181,  0.4719],
                [-0.6035,  1.3166,  0.2733]],
       
               [[ 0.8140, -0.0181,  0.4719],
                [-0.6035,  1.3166,  0.2733]],
       
               [[ 0.8140, -0.0181,  0.4719],
                [-0.6035,  1.3166,  0.2733]],
       
               [[ 0.8140, -0.0181,  0.4719],
                [-0.6035,  1.3166,  0.2733]]]),
    x: Batch(
           c: tensor([[[ 0.6333, -1.7667, -0.8784,  0.2224],
                       [-1.5923,  0.0482, -1.1485,  1.2433],
                       [ 0.1746,  0.1556,  0.4432,  0.4610]],
         

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 331 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.8140, -0.0181,  0.4719],
               [-0.6035,  1.3166,  0.2733],
               [ 0.8140, -0.0181,  0.4719],
               [-0.6035,  1.3166,  0.2733],
               [ 0.8140, -0.0181,  0.4719],
               [-0.6035,  1.3166,  0.2733],
               [ 0.8140, -0.0181,  0.4719],
               [-0.6035,  1.3166,  0.2733],
               [ 0.8140, -0.0181,  0.4719],
               [-0.6035,  1.3166,  0.2733],
               [ 0.8140, -0.0181,  0.4719],
               [-0.6035,  1.3166,  0.2733],
               [ 0.8140, -0.0181,  0.4719],
               [-0.6035,  1.3166,  0.2733],
               [ 0.8140, -0.0181,  0.4719],
               [-0.6035,  1.3166,  0.2733]]),
    x: Batch(
           c: tensor([[ 0.6333, -1.7667, -0.8784,  0.2224],
                      [-1.5923,  0.0482, -1.1485,  1.2433],
                      [ 0.1746,  0.1556,  0.4432,  0.4610],
                      [ 0.6333, -1.7667, -0.8784,  0.2224],
                      [-1.5923,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 314 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

359 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
